In [1]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

The `#!connect jupyter` feature is in preview. Please report any feedback or issues at https://github.com/dotnet/interactive/issues/new/choose.

Kernel added: #!pythonkernel

In [2]:
pip install ifcopenshell

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install numpy

In [ ]:
pip install lark

Note:
this code uses the IfcPatch utility

In [4]:
import sys
sys.path.append("C:/git/IfcOpenShell/src/ifcpatch/")
import ifcopenshell
import ifcpatch

filename = "C:/git/bim-net-interactive/samples/File.ifc";
# model = ifcopenshell.open("C:/git/bim-net-interactive/samples/File.ifc")


output = ifcpatch.execute({
    "input": filename,
    "file": ifcopenshell.open(filename),
    "recipe": "ExtractElements",
    "arguments": ["IfcWall"],
})
ifcpatch.write(output, "File_onlywalls.ifc")